In [1]:
cd ..

C:\Users\Nghia\PycharmProjects\ECML\Refactor


In [2]:
from pprint import pprint

import numpy as np
import tensorflow as tf
import time
from collections import Counter

from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest

from load_datasets import mnist
from metrics import metrics
from models.AE1SVM import AEOneClassSVM
from models.DEC import DEC
from models.RDA import RobustL21Autoencoder


c:\users\nghia\anaconda2\envs\tf36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
tf.set_random_seed(2018)

x_train, y_train, x_test, y_test = mnist(random_state=3)

counter = Counter(y_train)
print('Anomalies ratio:', 100*counter[-1]/(counter[1]+counter[-1]), '%')

c:\users\nghia\anaconda2\envs\tf36\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Anomalies ratio: 1.682935038707506 %


In [4]:
# Train conventional OCSVM
print('OCSVM-RBF')
libsvm = OneClassSVM(nu=0.14, verbose=True, shrinking=True)
t0 = time.time()
libsvm.fit(x_train)
print('Train time:', time.time() - t0)

t0 = time.time()
out_y = libsvm.predict(x_test)
print('Test time:', time.time() - t0)
pprint(metrics(y_test, out_y))

OCSVM-RBF
[LibSVM]Train time: 6.561444997787476
Test time: 2.3121464252471924
{'AUPRC': 0.9940264965014832,
 'AUROC': 0.8239267374186923,
 'Confusion matrix': array([[2535,  386],
       [  11,   39]], dtype=int64),
 'F1': 0.9273824766782514,
 'P@10': 0.983451536643026,
 'Precision': 0.9956794972505891,
 'Recall': 0.8678534748373845}


In [5]:
import matplotlib.pyplot as plt

anomalies = x_test[np.where(y_test == -1)]

for x in anomalies:
    plt.figure()
    plt.imshow(np.reshape(x, (28, 28)))
    print(libsvm.predict([x]))
    
    

[-1]
[1]
[-1]
[1]
[1]
[-1]
[1]
[-1]
[-1]
[-1]
[1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[1]


c:\users\nghia\anaconda2\envs\tf36\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[-1]
[1]
[-1]
[1]
[-1]
[1]
[1]
[-1]
[1]
[-1]


In [6]:
# Train conventional OCSVM
print('OCSVM-Linear')
libsvm = OneClassSVM(nu=0.14, verbose=True, shrinking=True, kernel='linear')
t0 = time.time()
libsvm.fit(x_train)
print('Train time:', time.time() - t0)

t0 = time.time()
out_y = libsvm.predict(x_test)
print('Test time:', time.time() - t0)
pprint(metrics(y_test, out_y))

OCSVM-Linear
[LibSVM]Train time: 6.6436638832092285
Test time: 2.760338068008423
{'AUPRC': 0.9845041919002815,
 'AUROC': 0.5402327969873332,
 'Confusion matrix': array([[2455,  466],
       [  38,   12]], dtype=int64),
 'F1': 0.906908016254156,
 'P@10': 0.9831138129010469,
 'Precision': 0.9847573204973927,
 'Recall': 0.8404655939746662}


In [7]:
# Train Isolation Forest
print('IsolationForest')
iforest = IsolationForest(contamination=0.15, verbose=1)
t0 = time.time()
iforest.fit(x_train)
print('Train time:', time.time() - t0)

t0 = time.time()
out_y = iforest.predict(x_test)
print('Test time:', time.time() - t0)
pprint(metrics(y_test, out_y))

IsolationForest


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.9s finished


Train time: 11.897631406784058
Test time: 6.182436943054199
{'AUPRC': 0.9914389532604087,
 'AUROC': 0.7474221157137967,
 'Confusion matrix': array([[2497,  424],
       [  18,   32]], dtype=int64),
 'F1': 0.9186902133922001,
 'P@10': 0.983451536643026,
 'Precision': 0.9928429423459244,
 'Recall': 0.8548442314275932}


In [8]:
batch_size = 16
autoencoder_layers = [784, 256, 64]

data_input = tf.placeholder(tf.float32, shape=[None, 784])

In [9]:
dec = DEC(dims=autoencoder_layers, n_clusters=5)
t0 = time.time()
dec.pretrain(x=x_train, epochs=5)
dec.compile(loss='kld')
y_pred = dec.fit(x_train, update_interval=10, batch_size=batch_size)
print('Train time:', time.time() - t0)

t0 = time.time()
scores = dec.cluster_score(x_test)
print('Test time:', time.time() - t0)
threshold = np.partition(scores.flatten(), int(counter[-1]))[int(counter[-1])]
out_y = [1 if s > 2*threshold else -1 for s in scores]
pprint(metrics(y_test, out_y))

...Pretraining...
Epoch 1/5
2971/2971 [==============================] - 2s 673us/step - loss: 0.0831
Epoch 2/5
2971/2971 [==============================] - 1s 208us/step - loss: 0.0502
Epoch 3/5
2971/2971 [==============================] - 1s 200us/step - loss: 0.0423
Epoch 4/5
2971/2971 [==============================] - 1s 207us/step - loss: 0.0352
Epoch 5/5
2971/2971 [==============================] - 1s 216us/step - loss: 0.0295
Pretraining time:  5.97187876701355
Update interval 10
Save interval 928.4375
Initializing cluster centers with k-means.
delta_label  0.0006731740154830024 < tol  0.001
Reached tolerance threshold. Stopping training.
Train time: 214.77528548240662
Test time: 0.3669748306274414
{'AUPRC': 0.9865547178653796,
 'AUROC': 0.6017904827114002,
 'Confusion matrix': array([[2172,  749],
       [  27,   23]], dtype=int64),
 'F1': 0.8484375,
 'P@10': 0.9831138129010469,
 'Precision': 0.9877216916780355,
 'Recall': 0.7435809654228004}


In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Robust Deep Autoencoder
    rae = RobustL21Autoencoder(sess=sess, lambda_=4, layers_sizes=autoencoder_layers, learning_rate=1e-3)
    t0 = time.time()
    L, S = rae.fit(x_train, sess=sess, inner_iteration=20, iteration=5, verbose=True, batch_size=batch_size)
    print('Train time:', time.time() - t0)

    t0 = time.time()
    L_test, S_test = rae.predict(x_test, sess=sess)
    print('Test time:', time.time() - t0)

    s_sum = np.linalg.norm(S, axis=1)
    s_sum_test = np.linalg.norm(S_test, axis=1)
    out_y = [1 if s == 0 else -1 for s in s_sum_test]
    pprint(metrics(y_test, out_y))

X shape:  (2971, 784)
L shape:  (2971, 784)
S shape:  (2971, 784)
Out iteration:  1
    iteration :  5 , cost :  0.03251317
    iteration :  10 , cost :  0.020535655
    iteration :  15 , cost :  0.015829932
    iteration :  20 , cost :  0.014072955
Out iteration:  2
    iteration :  5 , cost :  0.012962403
    iteration :  10 , cost :  0.012115979
    iteration :  15 , cost :  0.011359278
    iteration :  20 , cost :  0.010501235
Out iteration:  3
    iteration :  5 , cost :  0.009622929
    iteration :  10 , cost :  0.008739465
    iteration :  15 , cost :  0.007859766
    iteration :  20 , cost :  0.007146163
Out iteration:  4
    iteration :  5 , cost :  0.0066485642
    iteration :  10 , cost :  0.006248813
    iteration :  15 , cost :  0.0058979774
    iteration :  20 , cost :  0.0055942684
Out iteration:  5
    iteration :  5 , cost :  0.0053441143
    iteration :  10 , cost :  0.0050495365
    iteration :  15 , cost :  0.0048581357
    iteration :  20 , cost :  0.004667224
Trai

In [11]:
import matplotlib
import matplotlib.pyplot as plt

normal = x_test[np.where(y_test == 1)]
anomalies = x_test[np.where(y_test == -1)]

In [12]:
ae1svm = AEOneClassSVM(data_input, batch_size, 'test_normal', autoencoder_layers[1:], 0.4, 1e3, 3.0, 500,
                       autoencoder_activation='sigmoid',
                       full_op=tf.train.AdamOptimizer(1e-3),
                       svm_op=tf.train.AdamOptimizer(1e-4))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Autoencoder-OneclassSVM
    t0 = time.time()
    ae1svm.fit(sess, x_train, x_train, y_train, epochs_1=100, epochs_2=10)
    print('Train time:', time.time() - t0)

    t0 = time.time()
    out_y = ae1svm.predict(sess, x_test)
    print('Test time:', time.time() - t0)

    pprint(metrics(y_test, out_y))
    
    margin_a = ae1svm.decision_function(sess, anomalies).T
    rank_margin_a = np.argsort(margin_a)[0]
    for a in anomalies:
        plt.figure()
        gs = matplotlib.gridspec.GridSpec(1, 4)
        factor_0 = sess.run(ae1svm.gradient, feed_dict={data_input: [a]})[0]
        factor_0 = np.reshape(factor_0, (28, 28))
        plt.subplot(gs[0]).imshow(np.reshape(a, (28, 28)))
        plt.subplot(gs[1]).imshow(np.maximum(0, factor_0), cmap='Reds')
        plt.subplot(gs[2]).imshow(np.maximum(0, -factor_0), cmap='Reds')
        plt.subplot(gs[3]).imshow(factor_0, cmap='Reds')


Combined train
Epoch: 1 Loss: 0.1037149804884719 ( 2.0453670648002175e-05 x 1000.0 + 0.08326130561738734 ) AUROC: 0.5820267031838412
Epoch: 2 Loss: 0.07212217389353595 ( 1.4038584080660475e-05 x 1000.0 + 0.05808358977275118 ) AUROC: 0.49452242382745637
Epoch: 3 Loss: 0.051848353460075 ( 1.1537746924095705e-05 x 1000.0 + 0.04031060577361763 ) AUROC: 0.4875042793563848
Epoch: 4 Loss: 0.037015241470773466 ( 9.265131132444601e-06 x 1000.0 + 0.027750110915115648 ) AUROC: 0.4924409448818898
Epoch: 5 Loss: 0.026980677103603058 ( 8.072480258416824e-06 x 1000.0 + 0.01890819604771581 ) AUROC: 0.47762067785005136
Epoch: 6 Loss: 0.020042601758404156 ( 7.320624891855387e-06 x 1000.0 + 0.012721976861533233 ) AUROC: 0.48204382060938034
Epoch: 7 Loss: 0.015184345328819867 ( 6.756544343873893e-06 x 1000.0 + 0.008427801295909285 ) AUROC: 0.5041698048613489
Epoch: 8 Loss: 0.01104431897050722 ( 5.5675642161768965e-06 x 1000.0 + 0.005476754829563384 ) AUROC: 0.4990345771995892
Epoch: 9 Loss: 0.008690908195

In [ ]:
ae_only = AEOneClassSVM(data_input, batch_size, 'test_ae', autoencoder_layers[1:], 0.4, 1e3, 3.0, 500,
                       autoencoder_activation='sigmoid', ae_op=tf.train.AdamOptimizer(1e-3))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Autoencoder-OneclassSVM
    t0 = time.time()
    ae_only.fit_ae(sess, x_train, epochs=100)
    
    x_train_encoded = ae_only.encode(sess, x_train)
    x_test_encoded = ae_only.encode(sess, x_test)

    x_train_rff = ae_only.encode_rff(sess, x_train)
    x_test_rff = ae_only.encode_rff(sess, x_test)

In [ ]:
# Train conventional OCSVM
print('OCSVM-RBF')
libsvm = OneClassSVM(nu=0.14, verbose=True, shrinking=True)
t0 = time.time()
libsvm.fit(x_train_encoded)
print('Train time:', time.time() - t0)

t0 = time.time()
out_y = libsvm.predict(x_test_encoded)
print('Test time:', time.time() - t0)
pprint(metrics(y_test, out_y))

print('OCSVM-Linear')
libsvm = OneClassSVM(nu=0.14, verbose=True, shrinking=True, kernel='linear')
t0 = time.time()
libsvm.fit(x_train_encoded)
print('Train time:', time.time() - t0)

t0 = time.time()
out_y = libsvm.predict(x_test_encoded)
print('Test time:', time.time() - t0)
pprint(metrics(y_test, out_y))

In [ ]:
print('OCSVM-Linear on RFF')
libsvm = OneClassSVM(nu=0.14, verbose=True, shrinking=False, kernel='linear')
t0 = time.time()
libsvm.fit(x_train_rff)
print('Train time:', time.time() - t0)

t0 = time.time()
out_y = libsvm.predict(x_test_rff)
print('Test time:', time.time() - t0)
pprint(metrics(y_test, out_y))